In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import random
import statsmodels.api as sm
songs=pd.read_csv('data/tracks.csv')
songs=songs.sample(100000)

In [2]:
songs_for_linear_regression = songs.copy()
songs_for_linear_regression.set_index('id',inplace=True)

In [3]:
songs_for_linear_regression.dropna(inplace=True)

In [4]:
songs_for_linear_regression

,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
id,,,,,,,,,,,,,,,,,,,
4csDivBHqf1T0eZTGkJvHQ,隆重登場,24,257456,0,['Joey Yung'],['2zzKlxMsKTPMsZacZCPRNA'],2001-11-21,0.633,0.931,8,-3.300,0,0.0425,0.0223,0.000000,0.2220,0.719,137.964,4
3WMIiXsBUJuLI5FHyjhFU1,Evlampia'S,31,192360,0,['Giannis Giokarinis'],['1kJDew8PqYeDB4TT4QLMpI'],2007-10-23,0.821,0.766,5,-6.162,1,0.0612,0.2540,0.000023,0.0242,0.935,111.527,4
1HaVsdm2IqgleDPCXYscOk,Life's Highway,38,197240,0,['Steve Wariner'],['45S8rtgQSxj1a0tzCNhSeD'],1985-01-01,0.660,0.371,10,-14.991,1,0.0238,0.0973,0.000000,0.1550,0.555,99.308,4
2gZeqcysy8j1FrrBUVdIb8,Boş Yere,47,240227,0,['Sıla'],['5gFPi3KWXEwA9bLEO47Ow0'],2010-11-29,0.614,0.658,6,-6.373,0,0.1630,0.3190,0.000000,0.1450,0.629,175.874,4
7wrGhtWHTHET8HWSX1DAWM,Esse Cara,22,113560,0,['Cazuza'],['1PwOU6fFbmaGkK3wkbb8fU'],1989-02-17,0.497,0.267,3,-11.129,1,0.0294,0.9350,0.000963,0.1160,0.167,80.913,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0ffOVvcToUWmxHPODrLUZw,Together We Can Make Such Sweet Music,20,184786,0,['The Spinners'],['5fbhwqYYh4YwUoEs582mq5'],1970,0.488,0.502,6,-11.378,1,0.0252,0.2170,0.000000,0.2800,0.829,99.263,4
3GVLUpqENyG1pEMYSkt9Uq,Üzüntüyü Bırak Yaşamaya Bak,41,202947,0,['Gönül Turgut'],['3E2lHRV6WfUemnxBu8C8Rw'],2008-01-25,0.251,0.539,5,-6.669,0,0.0313,0.9020,0.000117,0.5830,0.507,98.262,4
2srPfcRXDcDrQOhufrWhDu,Las Locuras Mías,57,222044,0,['Silvestre Dangond'],['3OcvS8PzSGYMBvLdzY6g3e'],2020-10-22,0.794,0.836,4,-2.655,1,0.0511,0.3820,0.000000,0.1610,0.966,155.925,4


In [5]:
def drop_columns(dataframe,to_be_deleted):
    dataframe.drop(to_be_deleted, axis=1, inplace=True)
to_be_deleted = ['id_artists', 'artists','name','release_date'] 

In [6]:
songs_for_linear_regression['release_date'] = pd.to_datetime(songs_for_linear_regression['release_date'], errors='coerce')

In [7]:
songs_for_linear_regression['year'] = pd.DatetimeIndex(songs_for_linear_regression['release_date']).year
songs_for_linear_regression['month'] = pd.DatetimeIndex(songs_for_linear_regression['release_date']).month

In [8]:
import sklearn
from sklearn import preprocessing

def scaler(dataframe,to_be_scaled):
    scaler = sklearn.preprocessing.StandardScaler(copy = True)
    dataframe[to_be_scaled] = scaler.fit_transform(dataframe[to_be_scaled].to_numpy())
# to_be_scaled = ['popularity','danceability','energy','key','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo','time_signature']
to_be_scaled = ['loudness','duration_ms','tempo']

In [9]:
songs_for_linear_regression.columns

Index(['name', 'popularity', 'duration_ms', 'explicit', 'artists',
       'id_artists', 'release_date', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'time_signature', 'year', 'month'],
      dtype='object')

In [10]:
drop_columns(songs_for_linear_regression,to_be_deleted)#delete columns
songs_for_linear_regression

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,month
id,,,,,,,,,,,,,,,,,
4csDivBHqf1T0eZTGkJvHQ,24,257456,0,0.633,0.931,8,-3.300,0,0.0425,0.0223,0.000000,0.2220,0.719,137.964,4,2001,11
3WMIiXsBUJuLI5FHyjhFU1,31,192360,0,0.821,0.766,5,-6.162,1,0.0612,0.2540,0.000023,0.0242,0.935,111.527,4,2007,10
1HaVsdm2IqgleDPCXYscOk,38,197240,0,0.660,0.371,10,-14.991,1,0.0238,0.0973,0.000000,0.1550,0.555,99.308,4,1985,1
2gZeqcysy8j1FrrBUVdIb8,47,240227,0,0.614,0.658,6,-6.373,0,0.1630,0.3190,0.000000,0.1450,0.629,175.874,4,2010,11
7wrGhtWHTHET8HWSX1DAWM,22,113560,0,0.497,0.267,3,-11.129,1,0.0294,0.9350,0.000963,0.1160,0.167,80.913,4,1989,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0ffOVvcToUWmxHPODrLUZw,20,184786,0,0.488,0.502,6,-11.378,1,0.0252,0.2170,0.000000,0.2800,0.829,99.263,4,1970,1
3GVLUpqENyG1pEMYSkt9Uq,41,202947,0,0.251,0.539,5,-6.669,0,0.0313,0.9020,0.000117,0.5830,0.507,98.262,4,2008,1
2srPfcRXDcDrQOhufrWhDu,57,222044,0,0.794,0.836,4,-2.655,1,0.0511,0.3820,0.000000,0.1610,0.966,155.925,4,2020,10


In [11]:
songs_for_linear_regression

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,month
id,,,,,,,,,,,,,,,,,
4csDivBHqf1T0eZTGkJvHQ,24,257456,0,0.633,0.931,8,-3.300,0,0.0425,0.0223,0.000000,0.2220,0.719,137.964,4,2001,11
3WMIiXsBUJuLI5FHyjhFU1,31,192360,0,0.821,0.766,5,-6.162,1,0.0612,0.2540,0.000023,0.0242,0.935,111.527,4,2007,10
1HaVsdm2IqgleDPCXYscOk,38,197240,0,0.660,0.371,10,-14.991,1,0.0238,0.0973,0.000000,0.1550,0.555,99.308,4,1985,1
2gZeqcysy8j1FrrBUVdIb8,47,240227,0,0.614,0.658,6,-6.373,0,0.1630,0.3190,0.000000,0.1450,0.629,175.874,4,2010,11
7wrGhtWHTHET8HWSX1DAWM,22,113560,0,0.497,0.267,3,-11.129,1,0.0294,0.9350,0.000963,0.1160,0.167,80.913,4,1989,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0ffOVvcToUWmxHPODrLUZw,20,184786,0,0.488,0.502,6,-11.378,1,0.0252,0.2170,0.000000,0.2800,0.829,99.263,4,1970,1
3GVLUpqENyG1pEMYSkt9Uq,41,202947,0,0.251,0.539,5,-6.669,0,0.0313,0.9020,0.000117,0.5830,0.507,98.262,4,2008,1
2srPfcRXDcDrQOhufrWhDu,57,222044,0,0.794,0.836,4,-2.655,1,0.0511,0.3820,0.000000,0.1610,0.966,155.925,4,2020,10


In [12]:
def scale_whole_df_except_valence():
    valence_only = songs_for_linear_regression['valence']
    del songs_for_linear_regression["valence"]

    x = songs_for_linear_regression #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    songs_for_linear_regression[songs_for_linear_regression.columns] = min_max_scaler.fit_transform(songs_for_linear_regression[songs_for_linear_regression.columns].to_numpy())
    songs_for_linear_regression['valence'] = valence_only
scale_whole_df_except_valence()

In [13]:
songs_for_linear_regression

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,tempo,time_signature,year,month,valence
id,,,,,,,,,,,,,,,,,
4csDivBHqf1T0eZTGkJvHQ,0.247423,0.052453,0.0,0.641988,0.931,0.727273,0.884417,0.0,0.043769,0.022390,0.000000,0.222222,0.582621,0.8,0.834711,0.909091,0.719
3WMIiXsBUJuLI5FHyjhFU1,0.319588,0.038874,0.0,0.832657,0.766,0.454545,0.839775,1.0,0.063028,0.255020,0.000023,0.024224,0.470977,0.8,0.884298,0.818182,0.935
1HaVsdm2IqgleDPCXYscOk,0.391753,0.039892,0.0,0.669371,0.371,0.909091,0.702059,1.0,0.024511,0.097691,0.000000,0.155155,0.419377,0.8,0.702479,0.000000,0.555
2gZeqcysy8j1FrrBUVdIb8,0.484536,0.048859,0.0,0.622718,0.658,0.545455,0.836484,0.0,0.167868,0.320281,0.000000,0.145145,0.742714,0.8,0.909091,0.909091,0.629
7wrGhtWHTHET8HWSX1DAWM,0.226804,0.022438,0.0,0.504057,0.267,0.272727,0.762299,1.0,0.030278,0.938755,0.000963,0.116116,0.341695,0.8,0.735537,0.090909,0.167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0ffOVvcToUWmxHPODrLUZw,0.206186,0.037295,0.0,0.494929,0.502,0.545455,0.758415,1.0,0.025953,0.217871,0.000000,0.280280,0.419187,0.8,0.578512,0.000000,0.829
3GVLUpqENyG1pEMYSkt9Uq,0.422680,0.041083,0.0,0.254564,0.539,0.454545,0.831867,0.0,0.032235,0.905622,0.000117,0.583584,0.414960,0.8,0.892562,0.000000,0.507
2srPfcRXDcDrQOhufrWhDu,0.587629,0.045066,0.0,0.805274,0.836,0.363636,0.894478,1.0,0.052626,0.383534,0.000000,0.161161,0.658470,0.8,0.991736,0.818182,0.966


In [14]:
songs_for_linear_regression["key"] = songs_for_linear_regression["key"].astype("category")
songs_for_linear_regression = pd.get_dummies(songs_for_linear_regression, columns=["key"])

* As always when coverting to dummy variables, we will rename them to logical names. 
* In the following loop we assign rename a column to a key_x where x is the number accordingly

In [15]:
starting_key = 0 
for i in range (16, 28):
    songs_for_linear_regression = songs_for_linear_regression.rename(columns={songs_for_linear_regression.columns[i]: 'key_%d' % starting_key})
    starting_key = starting_key  + 1 

In [16]:
songs_for_linear_regression.head(5)

,popularity,duration_ms,explicit,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,...,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
id,,,,,,,,,,,,,,,,,,,,,
4csDivBHqf1T0eZTGkJvHQ,0.247423,0.052453,0.0,0.641988,0.931,0.884417,0.0,0.043769,0.022390,0.000000,...,0,0,0,0,0,0,1,0,0,0
3WMIiXsBUJuLI5FHyjhFU1,0.319588,0.038874,0.0,0.832657,0.766,0.839775,1.0,0.063028,0.255020,0.000023,...,0,0,0,1,0,0,0,0,0,0
1HaVsdm2IqgleDPCXYscOk,0.391753,0.039892,0.0,0.669371,0.371,0.702059,1.0,0.024511,0.097691,0.000000,...,0,0,0,0,0,0,0,0,1,0
2gZeqcysy8j1FrrBUVdIb8,0.484536,0.048859,0.0,0.622718,0.658,0.836484,0.0,0.167868,0.320281,0.000000,...,0,0,0,0,1,0,0,0,0,0
7wrGhtWHTHET8HWSX1DAWM,0.226804,0.022438,0.0,0.504057,0.267,0.762299,1.0,0.030278,0.938755,0.000963,...,0,1,0,0,0,0,0,0,0,0


In [17]:
# scaler(songs_for_linear_regression,to_be_scaled)

In [18]:
songs_for_linear_regression.columns

Index(['popularity', 'duration_ms', 'explicit', 'danceability', 'energy',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'tempo', 'time_signature', 'year', 'month', 'valence',
       'key_0', 'key_1', 'key_2', 'key_3', 'key_4', 'key_5', 'key_6', 'key_7',
       'key_8', 'key_9', 'key_10', 'key_11'],
      dtype='object')

We move our dependent variable to the last column

In [19]:
x = songs_for_linear_regression['valence']
del songs_for_linear_regression["valence"]
songs_for_linear_regression['valence'] = x

In [20]:
songs_for_linear_regression.columns[-1]

'valence'

# AND SO IT BEGINS

# CODE FOR FORWARD SELECTION

In [21]:
def process_subset(y, data, feature_set):
    X = data.loc[:, feature_set].values
    X = sm.add_constant(X)
    names = ['intercept']
    names.extend(feature_set)
    model = sm.OLS(y, X)
    model.data.xnames = names
    regr = model.fit()
    return regr

In [22]:
import itertools

def get_best_of_k(y, data, k):
    
    best_rsquared = 0
    best_model = None
    for comb in itertools.combinations(data.columns, k):
        regr = process_subset(y, data, comb)
        if regr.rsquared > best_rsquared:
            best_rsquared = regr.rsquared
            best_model = regr

    return best_model

In [23]:
def best_subset_selection(data, exog):
    best_model = None
    best_models = []
    y = data.loc[:, exog]
    endog = [ x for x in data.columns if x != exog ]
    X = data.loc[:, endog]

    for i in range(1, len(data.columns)):
        print(f'Finding the best model for {i} variable{"s" if i > 1 else ""}')
        model = get_best_of_k(y, X, i)
        if not best_model or model.rsquared_adj > best_model.rsquared_adj:
            best_model = model
        print(model.model.data.xnames[1:]) # get the variables minums the intercept
        best_models.append(model)

    print(f'Fitted {2**len(data.columns)} models')
    return best_model, best_models

In [24]:
def forward_add_variable(data, exog, selected, to_select):
    best_rsquared = 0
    best_model = None
    best_column = None
    y = data.loc[:, exog]
    
    for column in to_select:
        new_selected = selected + [column]
        regr = process_subset(y, data, new_selected)
        if regr.rsquared > best_rsquared:
            best_rsquared = regr.rsquared
            best_model = regr
            best_column = column
    
    return best_model, best_column

In [25]:
def forward_stepwise_selection(data, exog):

    best_models = []
    best_model = None
    selected = []
    to_select = [ x for x in data.columns if x != exog ]

    p = len(to_select) + 1

    for i in range(1, p):
        print(f'Finding the best model for {i} variable{"s" if i > 1 else ""}')
        model, best_column = forward_add_variable(data, exog, selected, to_select)
        selected.append(best_column)
        to_select.remove(best_column)
        if not best_model or model.rsquared_adj > best_model.rsquared_adj:
            best_model = model
        print(selected)
        best_models.append(model)
        print(model.rsquared_adj)
        
    print(f'Fitted {1 + p*(p+1)//2} models')
    return best_model, best_models

In [26]:
best_model, _ = forward_stepwise_selection(songs_for_linear_regression, 'valence')
print('Best overall model:', len(best_model.model.exog_names), best_model.model.exog_names)

Finding the best model for 1 variable
['danceability']
0.2794626174906245
Finding the best model for 2 variables
['danceability', 'energy']
0.3427334140250784
Finding the best model for 3 variables
['danceability', 'energy', 'year']
0.4224370210974885
Finding the best model for 4 variables
['danceability', 'energy', 'year', 'tempo']
0.433752080496683
Finding the best model for 5 variables
['danceability', 'energy', 'year', 'tempo', 'duration_ms']
0.4429005902980869
Finding the best model for 6 variables
['danceability', 'energy', 'year', 'tempo', 'duration_ms', 'acousticness']
0.4506510521003567
Finding the best model for 7 variables
['danceability', 'energy', 'year', 'tempo', 'duration_ms', 'acousticness', 'explicit']
0.4564771164003416
Finding the best model for 8 variables
['danceability', 'energy', 'year', 'tempo', 'duration_ms', 'acousticness', 'explicit', 'speechiness']
0.4601199337586228
Finding the best model for 9 variables
['danceability', 'energy', 'year', 'tempo', 'duration

In [27]:
best_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                valence   R-squared:                       0.470
Model:                            OLS   Adj. R-squared:                  0.470
Method:                 Least Squares   F-statistic:                     3699.
Date:                Wed, 23 Feb 2022   Prob (F-statistic):               0.00
Time:                        22:00:29   Log-Likelihood:                 25628.
No. Observations:               99987   AIC:                        -5.121e+04
Df Residuals:                   99962   BIC:                        -5.097e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
intercept            0.2715      0.011     25.493      0.000       0.251       0.292
danceability         0.8125      0.004    204.406      0.000       0.805       0.820
energy               0.5476      0.005    116.507      0.000       0.538       0.557
year                -0.3513      0.004    -78.734      0.000      -0.360      -0.343
tempo                0.2187      0.005     44.709      0.000       0.209       0.228
duration_ms         -0.9577      0.023    -40.828      0.000      -1.004      -0.912
acousticness         0.1121      0.003     42.418      0.000       0.107       0.117
explicit            -0.0783      0.003    -25.930      0.000      -0.084      -0.072
speechiness         -0.1212      0.004    -33.410      0.000      -0.128      -0.114
instrumentalness    -0.0781      0.002    -31.564      0.000      -0.083      -0.073
loudness            -0.3841      0.013    -29.890      0.000      -0.409      -0.359
mode                 0.0137      0.001     10.538      0.000       0.011       0.016
popularity          -0.0325      0.004     -8.182      0.000      -0.040      -0.025
key_1               -0.0186      0.002     -7.443      0.000      -0.023      -0.014
month               -0.0115      0.002     -6.792      0.000      -0.015      -0.008
key_8               -0.0149      0.003     -5.515      0.000      -0.020      -0.010
key_4               -0.0101      0.002     -4.315      0.000      -0.015      -0.005
key_5                0.0067      0.002      2.980      0.003       0.002       0.011
key_9                0.0043      0.002      2.065      0.039       0.000       0.008
liveness             0.0083      0.003      2.433      0.015       0.002       0.015
key_11              -0.0069      0.003     -2.700      0.007      -0.012      -0.002
key_6               -0.0063      0.003     -2.239      0.025      -0.012      -0.001
time_signature      -0.0116      0.006     -1.786      0.074      -0.024       0.001
key_2               -0.0036      0.002     -1.766      0.077      -0.008       0.000
key_3               -0.0049      0.003     -1.493      0.135      -0.011       0.002
==============================================================================
Omnibus:                      338.136   Durbin-Watson:                   2.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              340.669
Skew:                          -0.140   Prob(JB):                     1.06e-74
Kurtosis:                       2.946   Cond. No.                         85.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [28]:
best_model.coef_

AttributeError: 'OLSResults' object has no attribute 'coef'